In [ ]:
import warnings

warnings.filterwarnings(action="ignore")

# Data from POH of Tecnam P2006T

In [ ]:
import pandas as pd
import numpy as np

from stdatm import Atmosphere

results_p2006t = pd.read_csv("data/p2006t_data.csv")

altitude_training_points = results_p2006t["altitude [ft]"].to_numpy()
engine_rpm_training_points = results_p2006t["propeller rpm [1/min]"].to_numpy() * 2.43
map_training_points = results_p2006t["MAP [inHg]"].to_numpy()
power_rate_training_points = results_p2006t["power rate [-]"].to_numpy()
ff_training_points = results_p2006t["fuel flow [l/h]"].to_numpy()

sigma_training_points = (
    Atmosphere(altitude_training_points, altitude_in_feet=True).density / Atmosphere(0).density
)
rpm_ratio_training_points = engine_rpm_training_points / 5500.0

power_training_points = power_rate_training_points * 69.0  # In kW
fc_training_points = ff_training_points * 0.72  # in kg/h
sfc_training_points = fc_training_points / power_training_points * 1000.0  # in g/kWh
k_sfc_training_points = sfc_training_points / 285.0  # SFC at SL, MCP

displacement_volume = 1352.0  # In cm**3
displacement_volume /= 1e6

In [ ]:
print(sfc_training_points)
print(ff_training_points)
print(power_training_points)

For some reasons the SFC seems to be more or less constant at all point, which contradicts what was found in litterature and what is observed on the Pipistrel SW121.

In [ ]:
import plotly.express as px

pi1 = sigma_training_points
pi2 = map_training_points / 29.92
pi3 = rpm_ratio_training_points
pi4 = k_sfc_training_points

doe_k_sfc = np.c_[pi1, pi2, pi3, pi4]
doe_k_sfc = pd.DataFrame(doe_k_sfc, columns=["sigma", "MAP ratio", "RPM ratio", "SFC ratio"])

df = px.data.iris()
fig = px.scatter_matrix(doe_k_sfc, width=1200, height=800)
fig.show()

In [ ]:
# Import pyVPLM modules/functions
from pyvplm.addon.variablepowerlaw import regression_models, perform_regression

# Fit with 3rd order power-law model the obtained Pi DOE
model = regression_models(
    doe_k_sfc.values, elected_pi0="pi4", order=3, log_space=False, test_mode=True
)
perform_regression(doe_k_sfc.values, model, chosen_model=2)